# Value of Plays Using Run Expectancy

## The Run Expectancy Matrix

The **run expectancy matrix** is a fundamental sabermetrics concept that calculates the average number of runs expected to score for each game state (combination of baserunners and outs).

**Key concepts:**
- 8 possible base states (each of 3 bases can be empty or occupied: 2³ = 8)
- 3 possible out states (0, 1, or 2 outs)
- **24 total game states** (8 × 3 = 24)

## Runs Scored in the Remainder of the Inning

$$\text{runs}_{\text{roi}} = \text{runs}_{\text{Total in Inning}} - \text{runs}_{\text{So far in Inning}}$$

This calculates how many runs will be scored from the current game state until the end of the inning.

### Data Source

Using Retrosheet play-by-play data for the 2016 MLB season. Each row represents a single plate appearance with detailed information about the game state, players involved, and the outcome of the play.

In [10]:
import pandas as pd
fields = pd.read_csv('../data/fields.csv')
headers = fields['Header'].str.lower().tolist()

retro2016 = pd.read_csv('../data/all2016.csv', names=headers, low_memory=False)

In [11]:
# Calculate total runs scored before each plate appearance
retro2016['runs_before'] = retro2016['away_score_ct'] + retro2016['home_score_ct']

# Create unique identifier for each half inning (game_id + inning + batting team)
retro2016['half_inning'] = retro2016[['game_id', 'inn_ct', 'bat_home_id']].astype(str).agg(' '.join, axis=1)

In [12]:
# Calculate runs scored on each play (count runners who reached base > 3, meaning they scored)
retro2016['runs_scored'] = (
    (retro2016['bat_dest_id'] > 3).astype(int) + 
    (retro2016['run1_dest_id'] > 3).astype(int) + 
    (retro2016['run2_dest_id'] > 3).astype(int) + 
    (retro2016['run3_dest_id'] > 3).astype(int)
)

In [13]:
# Group by half inning and calculate totals for outs and runs
half_innings = retro2016.groupby('half_inning').agg({
    'event_outs_ct': 'sum',
    'runs_scored': 'sum',
    'runs_before': 'first'
}).reset_index()

# Rename columns
half_innings.columns = ['half_inning', 'outs_inning', 'runs_inning', 'runs_start']

# Calculate total runs at end of half inning
half_innings['max_runs'] = half_innings['runs_inning'] + half_innings['runs_start']

In [14]:
# Merge retro2016 with half_innings data to get inning totals
retro2016 = retro2016.merge(half_innings, on='half_inning', how='inner')

# Calculate runs scored in remainder of inning from current game state
retro2016['runs_roi'] = retro2016['max_runs'] - retro2016['runs_before']

## Creating the Matrix

Now that runs scored in the remainder of each inning (`runs_roi`) have been calculated, the run expectancy matrix can be built out.

**Creating the game state variable:**
- Combine baserunner positions into a 3-digit code
- Add the number of outs to create complete game states
- Examples:
  - "011 2" = runners on 2nd and 3rd with 2 outs
  - "100 0" = runner on 1st with 0 outs

In [15]:
import numpy as np

# Create bases code: 3-digit string indicating which bases are occupied or empty
retro2016['bases'] = (
    np.where(retro2016['base1_run_id'].notna() & (retro2016['base1_run_id'] != ''), '1', '0') +
    np.where(retro2016['base2_run_id'].notna() & (retro2016['base2_run_id'] != ''), '1', '0') +
    np.where(retro2016['base3_run_id'].notna() & (retro2016['base3_run_id'] != ''), '1', '0')
)

# Combine bases and outs to create game state
retro2016['state'] = retro2016['bases'] + ' ' + retro2016['outs_ct'].astype(str)

In [16]:
# Determine which bases are occupied after each play
retro2016['is_runner1'] = (
    (retro2016['run1_dest_id'] == 1) | (retro2016['bat_dest_id'] == 1)
).astype(int)

retro2016['is_runner2'] = (
    (retro2016['run1_dest_id'] == 2) | 
    (retro2016['run2_dest_id'] == 2) | 
    (retro2016['bat_dest_id'] == 2)
).astype(int)

retro2016['is_runner3'] = (
    (retro2016['run1_dest_id'] == 3) | 
    (retro2016['run2_dest_id'] == 3) |
    (retro2016['run3_dest_id'] == 3) | 
    (retro2016['bat_dest_id'] == 3)
).astype(int)

# Calculate outs and bases after the play
retro2016['new_outs'] = retro2016['outs_ct'] + retro2016['event_outs_ct']

# Create new bases code and state after the play
retro2016['new_bases'] = (
    retro2016['is_runner1'].astype(str) +
    retro2016['is_runner2'].astype(str) +
    retro2016['is_runner3'].astype(str)
)

retro2016['new_state'] = retro2016['new_bases'] + ' ' + retro2016['new_outs'].astype(str)

In [17]:
# Filter to plays where game state changed or runs were scored
changes2016 = retro2016[(retro2016['state'] != retro2016['new_state']) | 
                        (retro2016['runs_scored'] > 0)].copy()

# Filter to complete half innings only (exactly 3 outs recorded)
changes2016_complete = changes2016[changes2016['outs_inning'] == 3].copy()

In [18]:
# Calculate run expectancy for each bases/outs combination
erm_2016 = changes2016_complete.groupby(['bases', 'outs_ct']).agg(
    mean_run_value=('runs_roi', 'mean')
).reset_index()

# Display as matrix with outs as columns
erm_2016.pivot(
    index='bases', 
    columns='outs_ct', 
    values='mean_run_value'
).rename(columns=lambda x: f'Outs={x}')

outs_ct,Outs=0,Outs=1,Outs=2
bases,,,
000,0.498168,0.267903,0.106208
001,1.346979,0.936888,0.371953
010,1.133176,0.672856,0.312491
011,1.928788,1.357786,0.548168
100,0.857981,0.512134,0.220206
101,1.723270,1.195757,0.478245
110,1.444663,0.920621,0.413610
111,2.106105,1.537215,0.695272


### Comparing Run Expectancy Across Eras

To see how run expectancy values have changed over time, comparing the 2016 season values with 2002 season values.

In [20]:
# 2002 run expectancy values from Albert and Bennett (2003)
erm_2002 = pd.DataFrame({
    'OLD=0': [0.51, 1.40, 1.14, 1.96, 0.90, 1.84, 1.51, 2.33],
    'OLD=1': [0.27, 0.94, 0.68, 1.36, 0.54, 1.18, 0.94, 1.51],
    'OLD=2': [0.10, 0.36, 0.32, 0.63, 0.23, 0.52, 0.45, 0.78]
}, index=['000', '001', '010', '011', '100', '101', '110', '111'])

# Combine 2016 and 2002 values
out = erm_2016.pivot(
    index='bases',
    columns='outs_ct',
    values='mean_run_value'
).rename(columns=lambda x: f'NEW={x}')

# Concatenate with 2002 values
out = pd.concat([out, erm_2002], axis=1)

out

,NEW=0,NEW=1,NEW=2,OLD=0,OLD=1,OLD=2
000,0.498168,0.267903,0.106208,0.51,0.27,0.10
001,1.346979,0.936888,0.371953,1.40,0.94,0.36
010,1.133176,0.672856,0.312491,1.14,0.68,0.32
011,1.928788,1.357786,0.548168,1.96,1.36,0.63
100,0.857981,0.512134,0.220206,0.90,0.54,0.23
101,1.723270,1.195757,0.478245,1.84,1.18,0.52
110,1.444663,0.920621,0.413610,1.51,0.94,0.45
111,2.106105,1.537215,0.695272,2.33,1.51,0.78


### Stability Over Time

These run expectancy values have remained very stable over recent baseball history. This indicates there have been few changes in the average run scoring tendencies of MLB teams between 2002 and 2016.
